In [1]:
train_langevine = True

In [2]:
import argparse
import json
import logging
import random
from datetime import datetime
from importlib import import_module
from itertools import chain
from os.path import join, exists

import matplotlib.pyplot as plt
import torch
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
from torch.autograd import Variable

from new.aae.pcutil import plot_3d_point_cloud
#from utils.util import find_latest_epoch, prepare_results_dir, cuda_setup, setup_logging

cudnn.benchmark = True


In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        gain = torch.nn.init.calculate_gain('relu')
        torch.nn.init.xavier_uniform_(m.weight, gain)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.constant_(m.weight, 1)
        torch.nn.init.constant_(m.bias, 0)
    elif classname.find('Linear') != -1:
        gain = torch.nn.init.calculate_gain('relu')
        torch.nn.init.xavier_uniform_(m.weight, gain)
        if m.bias is not None:
            torch.nn.init.constant_(m.bias, 0)


In [4]:
def get_grad_norm(parameters, norm_type=2):
    total_norm = 0
    for p in parameters:
        param_norm = p.grad.data.norm(norm_type)
        total_norm += param_norm.item() ** norm_type
    total_norm = total_norm ** (1. / norm_type)
    return total_norm

In [5]:
# random.seed(1234)
# torch.manual_seed(1234)
# torch.cuda.manual_seed_all(1234)

In [6]:
results_dir = "results/"

In [7]:
device = torch.device("cuda")

In [8]:
dataset_name = "shapenet"

In [9]:
from new.shapenet import ShapeNetDataset

In [10]:
dataset = ShapeNetDataset(root_dir="shapenet",
                          classes=["chair"])
points_dataloader = DataLoader(dataset, batch_size=16,
                               shuffle=True,
                               num_workers=8,
                               drop_last=True, pin_memory=True)

In [11]:
if train_langevine:
    from new.models import NetWrapper
    
    net = NetWrapper().to(device)
    net.apply(weights_init)

else:
    from new.aae.aae import Generator, Encoder

    G =  Generator().to(device)
    E = Encoder().to(device)

    G.apply(weights_init)
    E.apply(weights_init)

In [12]:
from new.params import *
from new.utils import *

In [13]:
#
# Float Tensors
#
fixed_noise = torch.FloatTensor(16, z_dim)
fixed_noise.normal_(mean=0, std=0.2)
std_assumed = torch.tensor(0.2)

fixed_noise = fixed_noise.to(device)
std_assumed = std_assumed.to(device)

In [14]:
#
# Optimizers
#
optim_params = {
                "lr": 0.0005,
                "weight_decay": 0,
                "betas": [0.9, 0.999],
                "amsgrad": False
            }

if train_langevine:
    optE = torch.optim.Adam(net.netE.parameters(), lr = 1e-6)
    optG = torch.optim.Adam(net.netG.parameters(), **optim_params)
else:
    EG_optim = torch.optim.Adam(chain(E.parameters(), G.parameters()),
                        **optim_params)

In [15]:
from new.champfer_loss import ChamferLoss

reconstruction_loss = ChamferLoss().to(device)

In [16]:
verbose = True

In [ ]:
for epoch in range(400):
    start_epoch_time = datetime.now()

    if train_langevine:
        net.train()
    else:     
        G.train()
        E.train()

    total_loss = 0.0
    for i, point_data in enumerate(points_dataloader, 1):
#         if i > 1:
#             break

        X, _ = point_data
        X = X.to(device)

        # Change dim [BATCH, N_POINTS, N_DIM] -> [BATCH, N_DIM, N_POINTS]
        if X.size(-1) == 3:
            X.transpose_(X.dim() - 2, X.dim() - 1)

        if train_langevine:
            batch_num = X.shape[0]

            # Initialize chains
            z_g_0 = sample_p_0(n = batch_num,sig=e_init_sig, device = X.device)
            z_e_0 = sample_p_0(n = batch_num,sig=g_init_sig, device = X.device)
            
            # print("z_g_0 norm", torch.mean(torch.linalg.vector_norm(z_g_0, dim = 1)))
            # print("z_e_0 norm", torch.mean(torch.linalg.vector_norm(z_e_0, dim = 1)))

            # Langevin posterior and prior
            z_g_k = net(Variable(z_g_0), X, prior=False, verbose = verbose)
            z_e_k = net(Variable(z_e_0), prior=True, verbose = verbose)

            # reconstruction
            X_hat = net.netG(z_g_k.detach())
            loss_g = net.loss_fun(X_hat.transpose(1,2).contiguous(), X.transpose(1,2).contiguous())

            # energy prior
            en_neg = net.netE(z_e_k.detach()).mean() # TODO(nijkamp): why mean() here and in Langevin sum() over energy? constant is absorbed into Adam adaptive lr
            en_pos = net.netE(z_g_k.detach()).mean()
            loss_e = (en_pos - en_neg) / batch_num

            # Learn generator
            optG.zero_grad()
            loss_g.backward()
            
            #if args.g_is_grad_clamp:
            # grad_norm_g = get_grad_norm(net.netG.parameters())
            # torch.nn.utils.clip_grad_norm(net.netG.parameters(), 10)
            
            optG.step()
            
        
            optE.zero_grad()

            #if args.g_is_grad_clamp:
            # grad_norm_e = get_grad_norm(net.netE.parameters())
            # torch.nn.utils.clip_grad_norm(net.netE.parameters(), 10)

            loss_e.backward()
            # grad_norm_e = get_grad_norm(net.netE.parameters())
            # if args.e_is_grad_clamp:
            #    torch.nn.utils.clip_grad_norm_(net.netE.parameters(), args.e_max_norm)
            optE.step()


            if i % 30 == 0:
                print(f'[{epoch}: ({i})] '
                          f'loss_g: {loss_g.item():.4f} '
                          f'(loss_e: {loss_e.item(): .4f}'
                          f' Time: {datetime.now() - start_epoch_time}')


        else:
            codes, mu, logvar = E(X)
            X_rec = G(codes)

            loss_e = torch.mean(
                 0.05 *
                reconstruction_loss(X.permute(0, 2, 1) + 0.5,
                                    X_rec.permute(0, 2, 1) + 0.5))

            loss_kld = -0.5 * torch.mean(
                1 - 2.0 * torch.log(std_assumed) + logvar -
                (mu.pow(2) + logvar.exp()) / torch.pow(std_assumed, 2))

            loss_eg = loss_e + loss_kld
            EG_optim.zero_grad()
            E.zero_grad()
            G.zero_grad()

            loss_eg.backward()
            total_loss += loss_eg.item()
            EG_optim.step()

            if i % 30 == 0:
                print(f'[{epoch}: ({i})] '
                          f'Loss_EG: {loss_eg.item():.4f} '
                          f'(REC: {loss_e.item(): .4f}'
                          f' KLD: {loss_kld.item(): .4f})'
                          f' Time: {datetime.now() - start_epoch_time}')

    print(
        f'[{epoch}/{400}] '
        f'Loss_G: {total_loss / i:.4f} '
        f'Time: {datetime.now() - start_epoch_time}'
    )
        
    ################################### eval ######################################
    #
    # Save intermediate results
    #
    if train_langevine:
        net.eval()
        with torch.no_grad():
            fake = net.netG(fixed_noise).data.cpu().numpy()
            X_rec = X_hat.data.cpu().numpy()
            X = X.data.cpu().numpy()
    else:
        G.eval()
        E.eval()
        with torch.no_grad():
            fake = G(fixed_noise).data.cpu().numpy()
            codes, _, _ = E(X)
            X_rec = G(codes).data.cpu().numpy()
            X = X.data.cpu().numpy()

    for k in range(5):
        fig = plot_3d_point_cloud(X[k][0], X[k][1], X[k][2],
                                  in_u_sphere=True, show=False)
        fig.savefig(
            join(results_dir, 'samples', f'{epoch}_{k}_real.png'))
        plt.close(fig)

    for k in range(5):
        fig = plot_3d_point_cloud(fake[k][0], fake[k][1], fake[k][2],
                                  in_u_sphere=True, show=False,
                                  title=str(epoch))
        fig.savefig(
            join(results_dir, 'samples', f'{epoch:05}_{k}_fixed.png'))
        plt.close(fig)

    for k in range(5):
        fig = plot_3d_point_cloud(X_rec[k][0],
                                  X_rec[k][1],
                                  X_rec[k][2],
                                  in_u_sphere=True, show=False)
        fig.savefig(join(results_dir, 'samples',
                         f'{epoch}_{k}_reconstructed.png'))
        plt.close(fig)

Langevin posterior   1/ 40: LOSS G=   9.266 z_norm:  11.092
Langevin posterior  11/ 40: LOSS G=   4.651 z_norm:   9.866
Langevin posterior  21/ 40: LOSS G=   3.074 z_norm:   9.217
Langevin posterior  31/ 40: LOSS G=   2.559 z_norm:   8.958
Langevin posterior  40/ 40: LOSS G=   2.474 z_norm:   8.944
Langevin prior   1/ 80: energy=  19.595 z_norm:  11.074 z_grad_norm:   1.434
Langevin prior  11/ 80: energy=  17.606 z_norm:  11.021 z_grad_norm:   1.426
Langevin prior  21/ 80: energy=  15.759 z_norm:  10.942 z_grad_norm:   1.418
Langevin prior  31/ 80: energy=  13.948 z_norm:  10.889 z_grad_norm:   1.411
Langevin prior  41/ 80: energy=  12.299 z_norm:  10.817 z_grad_norm:   1.407
Langevin prior  51/ 80: energy=  10.413 z_norm:  10.765 z_grad_norm:   1.400
Langevin prior  61/ 80: energy=   8.859 z_norm:  10.701 z_grad_norm:   1.398
Langevin prior  71/ 80: energy=   6.953 z_norm:  10.653 z_grad_norm:   1.392
Langevin prior  80/ 80: energy=   5.305 z_norm:  10.615 z_grad_norm:   1.387
Langevi

Langevin posterior  11/ 40: LOSS G=   0.968 z_norm:  10.241
Langevin posterior  21/ 40: LOSS G=   0.910 z_norm:   9.512
Langevin posterior  31/ 40: LOSS G=   0.918 z_norm:   9.076
Langevin posterior  40/ 40: LOSS G=   0.904 z_norm:   8.877
Langevin prior   1/ 80: energy=  16.478 z_norm:  11.234 z_grad_norm:   1.475
Langevin prior  11/ 80: energy=  14.477 z_norm:  11.183 z_grad_norm:   1.469
Langevin prior  21/ 80: energy=  12.835 z_norm:  11.158 z_grad_norm:   1.465
Langevin prior  31/ 80: energy=  10.838 z_norm:  11.102 z_grad_norm:   1.459
Langevin prior  41/ 80: energy=   9.303 z_norm:  11.065 z_grad_norm:   1.453
Langevin prior  51/ 80: energy=   7.434 z_norm:  11.007 z_grad_norm:   1.451
Langevin prior  61/ 80: energy=   5.889 z_norm:  10.964 z_grad_norm:   1.446
Langevin prior  71/ 80: energy=   4.235 z_norm:  10.908 z_grad_norm:   1.441
Langevin prior  80/ 80: energy=   3.073 z_norm:  10.857 z_grad_norm:   1.436
Langevin posterior   1/ 40: LOSS G=   0.905 z_norm:  11.287
Langevi

Langevin posterior  11/ 40: LOSS G=   1.041 z_norm:  10.212
Langevin posterior  21/ 40: LOSS G=   0.823 z_norm:   9.505
Langevin posterior  31/ 40: LOSS G=   0.690 z_norm:   9.133
Langevin posterior  40/ 40: LOSS G=   0.673 z_norm:   9.009
Langevin prior   1/ 80: energy=  11.956 z_norm:  11.005 z_grad_norm:   1.425
Langevin prior  11/ 80: energy=  10.028 z_norm:  10.952 z_grad_norm:   1.421
Langevin prior  21/ 80: energy=   8.422 z_norm:  10.889 z_grad_norm:   1.419
Langevin prior  31/ 80: energy=   6.848 z_norm:  10.842 z_grad_norm:   1.413
Langevin prior  41/ 80: energy=   5.407 z_norm:  10.779 z_grad_norm:   1.405
Langevin prior  51/ 80: energy=   3.674 z_norm:  10.734 z_grad_norm:   1.399
Langevin prior  61/ 80: energy=   2.436 z_norm:  10.669 z_grad_norm:   1.396
Langevin prior  71/ 80: energy=   0.800 z_norm:  10.619 z_grad_norm:   1.395
Langevin prior  80/ 80: energy=  -0.993 z_norm:  10.578 z_grad_norm:   1.389
Langevin posterior   1/ 40: LOSS G=   1.230 z_norm:  11.086
Langevi

Langevin posterior  11/ 40: LOSS G=   0.667 z_norm:  10.088
Langevin posterior  21/ 40: LOSS G=   0.588 z_norm:   9.487
Langevin posterior  31/ 40: LOSS G=   0.555 z_norm:   9.160
Langevin posterior  40/ 40: LOSS G=   0.534 z_norm:   9.045
Langevin prior   1/ 80: energy=  16.091 z_norm:  11.257 z_grad_norm:   1.467
Langevin prior  11/ 80: energy=  14.108 z_norm:  11.212 z_grad_norm:   1.463
Langevin prior  21/ 80: energy=  12.461 z_norm:  11.154 z_grad_norm:   1.460
Langevin prior  31/ 80: energy=  10.335 z_norm:  11.100 z_grad_norm:   1.457
Langevin prior  41/ 80: energy=   8.169 z_norm:  11.044 z_grad_norm:   1.457
Langevin prior  51/ 80: energy=   6.254 z_norm:  10.987 z_grad_norm:   1.450
Langevin prior  61/ 80: energy=   4.595 z_norm:  10.932 z_grad_norm:   1.446
Langevin prior  71/ 80: energy=   3.181 z_norm:  10.887 z_grad_norm:   1.441
Langevin prior  80/ 80: energy=   1.421 z_norm:  10.851 z_grad_norm:   1.442
Langevin posterior   1/ 40: LOSS G=   0.834 z_norm:  11.239
Langevi

Langevin posterior  11/ 40: LOSS G=   0.750 z_norm:   9.962
Langevin posterior  21/ 40: LOSS G=   0.658 z_norm:   9.299
Langevin posterior  31/ 40: LOSS G=   0.612 z_norm:   8.967
Langevin posterior  40/ 40: LOSS G=   0.575 z_norm:   8.910
Langevin prior   1/ 80: energy=  15.062 z_norm:  11.472 z_grad_norm:   1.431
Langevin prior  11/ 80: energy=  12.992 z_norm:  11.402 z_grad_norm:   1.423
Langevin prior  21/ 80: energy=  11.247 z_norm:  11.340 z_grad_norm:   1.417
Langevin prior  31/ 80: energy=   9.741 z_norm:  11.284 z_grad_norm:   1.413
Langevin prior  41/ 80: energy=   7.975 z_norm:  11.237 z_grad_norm:   1.410
Langevin prior  51/ 80: energy=   6.223 z_norm:  11.181 z_grad_norm:   1.402
Langevin prior  61/ 80: energy=   4.644 z_norm:  11.130 z_grad_norm:   1.403
Langevin prior  71/ 80: energy=   2.887 z_norm:  11.072 z_grad_norm:   1.400
Langevin prior  80/ 80: energy=   1.420 z_norm:  11.029 z_grad_norm:   1.399
Langevin posterior   1/ 40: LOSS G=   1.341 z_norm:  11.161
Langevi

...

Langevin posterior  11/ 40: LOSS G=   0.383 z_norm:   9.993
Langevin posterior  21/ 40: LOSS G=   0.352 z_norm:   9.351
Langevin posterior  31/ 40: LOSS G=   0.342 z_norm:   9.169
Langevin posterior  40/ 40: LOSS G=   0.337 z_norm:   9.141
Langevin prior   1/ 80: energy=  19.791 z_norm:  11.290 z_grad_norm:   1.552
Langevin prior  11/ 80: energy=  17.438 z_norm:  11.234 z_grad_norm:   1.545
Langevin prior  21/ 80: energy=  15.545 z_norm:  11.182 z_grad_norm:   1.543
Langevin prior  31/ 80: energy=  13.445 z_norm:  11.122 z_grad_norm:   1.541
Langevin prior  41/ 80: energy=  11.397 z_norm:  11.065 z_grad_norm:   1.533
Langevin prior  51/ 80: energy=   9.482 z_norm:  11.012 z_grad_norm:   1.532
Langevin prior  61/ 80: energy=   7.432 z_norm:  10.963 z_grad_norm:   1.529
Langevin prior  71/ 80: energy=   5.575 z_norm:  10.921 z_grad_norm:   1.525
Langevin prior  80/ 80: energy=   4.032 z_norm:  10.882 z_grad_norm:   1.521
Langevin posterior   1/ 40: LOSS G=   0.408 z_norm:  10.996
Langevi

Langevin posterior  11/ 40: LOSS G=   0.383 z_norm:  10.020
Langevin posterior  21/ 40: LOSS G=   0.375 z_norm:   9.477
Langevin posterior  31/ 40: LOSS G=   0.372 z_norm:   9.335
Langevin posterior  40/ 40: LOSS G=   0.373 z_norm:   9.390
Langevin prior   1/ 80: energy=  13.868 z_norm:  11.206 z_grad_norm:   1.616
Langevin prior  11/ 80: energy=  11.657 z_norm:  11.139 z_grad_norm:   1.614
Langevin prior  21/ 80: energy=   9.590 z_norm:  11.090 z_grad_norm:   1.614
Langevin prior  31/ 80: energy=   7.369 z_norm:  11.023 z_grad_norm:   1.617
Langevin prior  41/ 80: energy=   5.081 z_norm:  10.968 z_grad_norm:   1.613
Langevin prior  51/ 80: energy=   2.857 z_norm:  10.907 z_grad_norm:   1.613
Langevin prior  61/ 80: energy=   0.732 z_norm:  10.862 z_grad_norm:   1.608
Langevin prior  71/ 80: energy=  -1.302 z_norm:  10.816 z_grad_norm:   1.609
Langevin prior  80/ 80: energy=  -3.413 z_norm:  10.776 z_grad_norm:   1.605
Langevin posterior   1/ 40: LOSS G=   0.434 z_norm:  11.173
Langevi

Langevin posterior  11/ 40: LOSS G=   0.443 z_norm:   9.857
Langevin posterior  21/ 40: LOSS G=   0.419 z_norm:   9.265
Langevin posterior  31/ 40: LOSS G=   0.408 z_norm:   9.038
Langevin posterior  40/ 40: LOSS G=   0.399 z_norm:   9.046
Langevin prior   1/ 80: energy=  10.784 z_norm:  11.314 z_grad_norm:   1.561
Langevin prior  11/ 80: energy=   8.808 z_norm:  11.266 z_grad_norm:   1.557
Langevin prior  21/ 80: energy=   7.146 z_norm:  11.213 z_grad_norm:   1.551
Langevin prior  31/ 80: energy=   5.250 z_norm:  11.171 z_grad_norm:   1.543
Langevin prior  41/ 80: energy=   2.908 z_norm:  11.121 z_grad_norm:   1.541
Langevin prior  51/ 80: energy=   0.892 z_norm:  11.067 z_grad_norm:   1.536
Langevin prior  61/ 80: energy=  -1.120 z_norm:  11.010 z_grad_norm:   1.535
Langevin prior  71/ 80: energy=  -2.931 z_norm:  10.956 z_grad_norm:   1.532
Langevin prior  80/ 80: energy=  -4.532 z_norm:  10.912 z_grad_norm:   1.529
Langevin posterior   1/ 40: LOSS G=   0.402 z_norm:  11.180
Langevi

Langevin posterior  11/ 40: LOSS G=   0.352 z_norm:  10.096
Langevin posterior  21/ 40: LOSS G=   0.338 z_norm:   9.554
Langevin posterior  31/ 40: LOSS G=   0.333 z_norm:   9.372
Langevin posterior  40/ 40: LOSS G=   0.326 z_norm:   9.383
Langevin prior   1/ 80: energy=   7.913 z_norm:  11.261 z_grad_norm:   1.579
Langevin prior  11/ 80: energy=   5.989 z_norm:  11.226 z_grad_norm:   1.574
Langevin prior  21/ 80: energy=   4.229 z_norm:  11.175 z_grad_norm:   1.569
Langevin prior  31/ 80: energy=   2.154 z_norm:  11.126 z_grad_norm:   1.566
Langevin prior  41/ 80: energy=   0.296 z_norm:  11.064 z_grad_norm:   1.560
Langevin prior  51/ 80: energy=  -1.917 z_norm:  11.014 z_grad_norm:   1.556
Langevin prior  61/ 80: energy=  -4.028 z_norm:  10.965 z_grad_norm:   1.554
Langevin prior  71/ 80: energy=  -5.984 z_norm:  10.919 z_grad_norm:   1.552
Langevin prior  80/ 80: energy=  -8.096 z_norm:  10.877 z_grad_norm:   1.550
Langevin posterior   1/ 40: LOSS G=   0.403 z_norm:  11.311
Langevi

Langevin posterior   1/ 40: LOSS G=   0.430 z_norm:  11.366
Langevin posterior  11/ 40: LOSS G=   0.360 z_norm:  10.276
Langevin posterior  21/ 40: LOSS G=   0.331 z_norm:   9.776
Langevin posterior  31/ 40: LOSS G=   0.322 z_norm:   9.553
Langevin posterior  40/ 40: LOSS G=   0.311 z_norm:   9.657
Langevin prior   1/ 80: energy=  15.732 z_norm:  11.171 z_grad_norm:   1.594
Langevin prior  11/ 80: energy=  13.912 z_norm:  11.133 z_grad_norm:   1.596
Langevin prior  21/ 80: energy=  11.643 z_norm:  11.065 z_grad_norm:   1.595
Langevin prior  31/ 80: energy=   9.522 z_norm:  10.993 z_grad_norm:   1.592
Langevin prior  41/ 80: energy=   7.918 z_norm:  10.950 z_grad_norm:   1.588
Langevin prior  51/ 80: energy=   5.647 z_norm:  10.898 z_grad_norm:   1.584
Langevin prior  61/ 80: energy=   3.722 z_norm:  10.849 z_grad_norm:   1.584
Langevin prior  71/ 80: energy=   1.594 z_norm:  10.798 z_grad_norm:   1.583
Langevin prior  80/ 80: energy=  -0.529 z_norm:  10.751 z_grad_norm:   1.579
Langevi

In [ ]:
a = torch.randn(16,128)

In [ ]:
torch.mean(torch.linalg.vector_norm(a, dim = 1))

In [ ]:
batch_size